In [271]:
import os
import json
cwd = os.getcwd();

In [272]:
mode="toSmeg"
modelName = "slab"
outputOBJ = ""
outputSMEG = ""
#(x,y,z,face):(smeg index, obj index)
vertexes = {}
#(x,y,z):obj index
uniqueVertexes = {}

#(x,y,z,face):(smeg index, obj index)
normals = {}
#(x,y,z):obj index
uniqueNormals= {}

#(u,v,face):(smeg index, obj index)
uv = {}
#(x,y,z):obj index
uniqueUV = {}

#(smeg-point1,smeg-point2,smeg-point3):(obj-point1,obj-point2,obj-point3)
faces = {}
faceTextures = {}
#(face,localIndex),remappedIndex
faceVertexMap = {}
faceUVMap = {}
faceNormalMap = {}
groupTextures = {-1:0}
if(mode=="toOBJ"):
    filename="test_inputs/"+modelName+".smeg"
else:
    filename="test_inputs/"+modelName+".obj"
#fileName = input("Input File: ");


In [273]:
def smeg_to_obj(modelFile):
    global modelName
    global outputOBJ
    try: 
        inputFile = open(modelFile, "r");
        modelData = json.load(inputFile);
        outputOBJ = outputOBJ + "mtllib " + modelName + ".mtl\n"
        outputOBJ = outputOBJ + "o " + modelName + "\n"
        j = 0
        for f in modelData['faces']:
            i = 0
            for v in f['vertices']:
                vertexes.update({(v[0],v[1],v[2],j): (i,i)})
                uniqueVertexes.update({(v[0],v[1],v[2]):""})
    
                normals.update({(v[3],v[4],v[5],j,i): (i,i)})
                uniqueNormals.update({(v[3],v[4],v[5]): ""})
    
                uv.update({(v[6],v[7],j): (i,i)})
                uniqueUV.update({(v[6],v[7]): ""})
                i+=1
            for t in f['triangles']:
                faces.update({(t[0],t[1],t[2],j):(t[0],t[1],t[2])})
            faceTextures.update({j:f['texture']})
            j+=1
        
        #index uniqueVertexes and put them in the output file
        i = 1
        for v in uniqueVertexes:
            uniqueVertexes.update({v:i})
            outputOBJ += "v "+ str(v[0]) + " " + str(v[1]) + " "+ str(v[2]) + "\n"
            i+=1
            
        #index uniqueUVs and put them in the output file
        i = 1
        for u in uniqueUV:
            uniqueUV.update({u:i})
            outputOBJ += "vt "+ str(u[0]) + " " + str(u[1]) + "\n"
            i+=1
          
        #index uniqueNormals and put them in the output file
        i = 1
        for n in uniqueNormals:
            uniqueNormals.update({n:i})
            outputOBJ += "vn "+ str(n[0]) + " " + str(n[1]) + " "+ str(n[2]) + "\n"
            i+=1
                
        #matches smeg vertices to the obj unique vertices index
        #print("vertexes")#v
        for v in vertexes:
            vertexes.update({v: (vertexes[v][0],uniqueVertexes[(v[0],v[1],v[2])])})
            faceVertexMap.update({(v[3],vertexes[v][0]):vertexes[v][1]})
            #print(v," | ", vertexes[v])
        
        #matches smeg uvs to the obj unique uv indexs
        #print("\nuv")
        for u in uv:
            uv.update({u: (uv[u][0],uniqueUV[(u[0],u[1])])})
            faceUVMap.update({(u[2],uv[u][0]):uv[u][1]})
            #print(u," | ", uv[u]) 
            
        #matches smeg normals to the obj unique normal indexs    
        #print("\nnormals")#
        for n in normals:
            normals.update({n: (normals[n][0],uniqueNormals[(n[0],n[1],n[2])])})
            faceNormalMap.update({(n[3],normals[n][0]):normals[n][1]})
            #print(n," | ", normals[n])    
        
        #set material to none, and smooth shading to false
        outputOBJ += "usemtl None\ns off\n"
            
        #print(str(normals) + "\n") #vn
        #print(str(uv) + "\n") #vt
        #print(str(faces) + "\n") #f
        #print(faceNormalMap)
        
        #add the faces to the obj file, adding in groups based on the smeg groups and pairing the texted used
        faceGroup = -1
        for f in faces:
            faces.update({f:(faceVertexMap[(f[3],f[0])],faceVertexMap[(f[3],f[1])],faceVertexMap[(f[3],f[2])])})
            x = str(faceVertexMap[(f[3],f[0])]) + "/" + str(faceUVMap[(f[3],f[0])]) + "/" + str(faceNormalMap[(f[3],f[0])])
            y = str(faceVertexMap[(f[3],f[1])]) + "/" + str(faceUVMap[(f[3],f[1])]) + "/" + str(faceNormalMap[(f[3],f[1])])
            z = str(faceVertexMap[(f[3],f[2])]) + "/" + str(faceUVMap[(f[3],f[2])]) + "/" + str(faceNormalMap[(f[3],f[2])])
            if(faceGroup != f[3]): 
                faceGroup = f[3]
                outputOBJ += "g face"+str(faceGroup)+"-texture:"+str(faceTextures[f[3]])+"\n"
            #x = str(faceVertexMap[(f[3],f[0])]+1) + "//" + str(faceNormalMap[(f[3],f[0])]+1)
            #y = str(faceVertexMap[(f[3],f[1])]+1) + "//" + str(faceNormalMap[(f[3],f[1])]+1)
            #z = str(faceVertexMap[(f[3],f[2])]+1) + "//" + str(faceNormalMap[(f[3],f[2])]+1)
            outputOBJ += "f " + x + " " + y + " " + z +"\n"
        #print(outputOBJ)
        output = open("test_outputs/"+modelName+".obj","w")
        output.write(outputOBJ)
        output.close()
        
        
        inputFile.close()
    except IOError:
        print("File not accessible");  

In [274]:

def obj_to_smeg(modelFile):
    global modelName
    global outputSMEG
    group = -1
    try:  
        inputFile = open(modelFile, "r");
        inputLines = inputFile.readlines();
        v=1
        n=1
        u=1
        f=0
        for line in inputLines:
            tokens = line.split(' ')
            if(tokens[0]=='#'):
                #print("Comment")
                continue
            elif(tokens[0]=='mtllib'):
                #print("Materials")
                continue
            elif(tokens[0]=='o'):
                #print("Object Name")
                continue
            elif(tokens[0]=='v'):
                #print("Vertex")
                vertexes.update({v:[tokens[1],tokens[2],tokens[3].replace('\n','')]})
                v += 1
            elif(tokens[0]=='vt'):
                #print("UV map vertex")
                uv.update({u:[tokens[1],tokens[2].replace('\n','')]})
                u += 1
            elif(tokens[0]=='vn'):
                #print("Normal")
                normals.update({n:[tokens[1],tokens[2],tokens[3].replace('\n','')]})
                n += 1
            elif(tokens[0]=='usemtl'):
                #print("Material to use")
                continue
            elif(tokens[0]=='s'):
                #print("Smooth Shading")
                continue
            elif(tokens[0]=='f'):
                #print("Face")
                x = tokens[1].split('/')
                y = tokens[2].split('/')
                z = tokens[3].replace('\n','').split('/')
                faces.update({f:(group,[x,y,z])})
                f += 1
            elif(tokens[0]=='g'):
                #print("Group")
                texture = tokens[1].replace('face','').replace('texture:','').replace('\n','').split('-')
                groupTextures.update({int(texture[0]):int(texture[1])})
                group += 1
            #print(tokens)
        #print("Vertexes")
        #print(vertexes)
        
        #print("\nNormals")
        #print(normals)
        
        #print("\nUVs")
        #print(uv)
        
        #print("\nFaces")
        #print(faces)
        
        #print("\n Group textures")
        #print(groupTextures)
        
        outputSMEG = "{\"bounds\":[-0.5,-0.5,-0.5,0.5,0.5,0.5],\n"
        outputSMEG += "\"faces\":["
        
        
        faceTriangles = []
        lastGroup = faces[0][0]
        faceString=""
        i = 0
        localFaceMap = {}
        currentGroup = -1
        for g in groupTextures:
            currentGroup = g
            for f in faces:
                if(faces[f][0]==currentGroup):
                    faceTriangles[-1].append(faces[f][1])
            faceTriangles.append([])       
        i = 0
        group = -1
        for f in faceTriangles:
            faceVertices=[]
            localFaceMap = {}
                            #x,y,z, nz,ny,nz, ux,uv
            expandedVertex = [0,0,0,0,0,0,0,0]
            singleFaceTriangles = [] 
            for tri in f:
                localTri = []
                for v in tri:
                    expandedVertex[0] = float(vertexes[int(v[0])][0])
                    expandedVertex[1] = float(vertexes[int(v[0])][1])
                    expandedVertex[2] = float(vertexes[int(v[0])][2])
                    expandedVertex[3] = float(normals[int(v[2])][0])
                    expandedVertex[4] = float(normals[int(v[2])][1])
                    expandedVertex[5] = float(normals[int(v[2])][2])
                    expandedVertex[6] = float(uv[int(v[1])][0])
                    expandedVertex[7] = float(uv[int(v[1])][1])
                    addedValue = localFaceMap.setdefault(tuple(expandedVertex),i)
                    localTri.append(addedValue)
                    if(addedValue == i):
                        i+=1 
                singleFaceTriangles.append(localTri)
            
            group += 1
            
            for f in localFaceMap:
                faceVertices.append(list(f))
            if(len(faceVertices)>0):
                faceString=["{\"texture\":"+ str(groupTextures[group])+"," +"\"vertices\":"+str(faceVertices)+",\n"+"\"triangles\":"+str(singleFaceTriangles)+"}"]
                outputSMEG+=faceString
            else:
                print(",")
        outputSMEG += "],\n\"boxes\":[[-0.5,-0.5,-0.5,0.5,0.5,0.5]]}"
        print(outputSMEG)
        inputFile.close()
    except IOError:
        print("File not accessible");  

In [275]:

if(mode=="toOBJ"):
    smeg_to_obj(filename)
else:
    obj_to_smeg(filename)

,
{"bounds":[-0.5,-0.5,-0.5,0.5,0.5,0.5],
"faces":[{"texture":1,"vertices":[[0.5, 0.0, 0.5, -0.0, -0.0, 1.0, 1.0, 0.5], [-0.5, -0.5, 0.5, -0.0, -0.0, 1.0, 0.0, 1.0], [0.5, -0.5, 0.5, -0.0, -0.0, 1.0, 1.0, 1.0], [-0.5, 0.0, 0.5, -0.0, -0.0, 1.0, 0.0, 0.5]],
"triangles":[[0, 1, 2], [1, 0, 3]]},
{"texture":1,"vertices":[[-0.5, 0.0, -0.5, -1.0, 0.0, 0.0, 0.0, 0.5], [-0.5, -0.5, 0.5, -1.0, 0.0, 0.0, 1.0, 1.0], [-0.5, 0.0, 0.5, -1.0, 0.0, 0.0, 1.0, 0.5], [-0.5, -0.5, -0.5, -1.0, 0.0, 0.0, 0.0, 1.0]],
"triangles":[[4, 5, 6], [5, 4, 7]]},
{"texture":3,"vertices":[[0.5, 0.0, 0.5, 0.0, 1.0, 0.0, 1.0, 1.0], [-0.5, 0.0, -0.5, 0.0, 1.0, 0.0, 0.0, 0.0], [-0.5, 0.0, 0.5, 0.0, 1.0, 0.0, 0.0, 1.0], [0.5, 0.0, -0.5, 0.0, 1.0, 0.0, 1.0, 0.0]],
"triangles":[[8, 9, 10], [9, 8, 11]]},
{"texture":1,"vertices":[[0.5, -0.5, -0.5, 1.0, 0.0, 0.0, 1.0, 1.0], [0.5, 0.0, 0.5, 1.0, 0.0, 0.0, 0.0, 0.5], [0.5, -0.5, 0.5, 1.0, 0.0, 0.0, 0.0, 1.0], [0.5, 0.0, -0.5, 1.0, 0.0, 0.0, 1.0, 0.5]],
"triangles":[[12, 13, 14], [